# Class 2 - Types, Functions, and Object Oriented Features

## Types

Types in Julia come in several flavors.  For example, the following types may look somewhat familiar

In [ ]:
@show typeof(4)
@show typeof(3.2)
;

These types hold actual data.  Every type in Julia has a "super" type. The type structure is like a tree - Any is the root.

In [ ]:
@show super(Float64)
@show super(AbstractFloat)
@show super(Real)
@show super(Number)
@show super(Any)
;

You can query whether one type is a descendant of another using "<:"

In [ ]:
@show Float64 <: AbstractFloat
@show Float64 <: Any
@show Float64 <: Integer
;

Abstract types are nodes on the tree, but you never instantiate them (we'll talk about why you might want abstract types in functions)

In [ ]:
abstract cme257abstract
@show super(cme257abstract)
;

When you create a type, you can specify who its parent is.

In [ ]:
type cme257int <: cme257abstract # <: denotes "child of"
    x::Int64 # :: tells us exactly what type x should be
end
type cme257float <: cme257abstract
    x::Float64
end
@show super(cme257int)
@show super(cme257float)
;

To instantiate a member of the type:

In [ ]:
y = cme257int(4)
y.x

Types can be parameterized (similar to C++ templates).  The parameterized type can be inferred by the type of the arguments in the constructor, or made explicit.

In [ ]:
type cme257par{T} <: cme257abstract
    val::T 
end
;

In [ ]:
@show y = cme257par(3.5)
@show z = cme257par{Float64}(3)
;

Immutable types don't allow you to change data once you have instantiated the type.  This restriction gan give you better performance.

In [ ]:
immutable cme257immutable <: cme257abstract
   x 
end

a = cme257immutable(5)
@show a.x
@show a.x = 6
;

## Break for Worksheet 1

## Functions

Functions map a tuple of arguments to an output. For example, the following function maps two inputs to their sum.

In [ ]:
function cme257_sum(x, y)
    return x + y
end
@show z = cme257_sum(1,2)
;

Julia uses multiple dispatch in functions.  This means you can define two functions with the same name that will behave differently based on the type of the input.  A definition of one possible behavior for a function is [called a method](docs.julialang.org/en/latest/manual/methods).

In [ ]:
# this may not work if you have a 32-but architecture
function yell_my_type(x::Int64)
    println("I'M AN INT64!")
end
function yell_my_type(x::Float64)
    println("I'M A FLOAT64!")
end
yell_my_type(3)
yell_my_type(3.4)
;

If you've only defined your function for particular types, you may see an error like this:

In [ ]:
yell_my_type(3//4)

Errors can be bad if you expect the function to behave a certain way, or good if they indicate that someone is using your function incorrectly.  If you want the same behavior for all types that share an ancestor, you can use "<:"

In [ ]:
function yell_my_type{T <: Number}(x::T)
    println("I'M A NUMBER!")
end
yell_my_type(3//4)
yell_my_type(3.4) # note that the more specific function declaration is used
;

If you want to try to cover all possible inputs, you can just leave off type annotations.  Your more specific methods will still be used for the relevant inputs.

In [ ]:
function yell_my_type(x)
    println("I DON'T KNOW MY TYPE!")
end
yell_my_type(yell_my_type) # everything in Julia has a type.  Even functions.
yell_my_type(3//4)
yell_my_type(3.4)
;

You can also add new methods to Julia's built in functions:

In [ ]:
import Base.+
function +(a::cme257int, b::cme257int)
    return cme257int((a.x+b.x))
end
a = cme257int(3)
b = cme257int(4)
@show a + b
;

## Break for Worksheet 2

## More on Types and Functions

Sometimes you may wish to have optional arguments to a function that default to certain values.

In [ ]:
function say(str="hello")
    println("$str")
end
say()
say("goodbye")
;

What if you have defined a special integer type (such as cme257int), and want to add a method to "/" to implement division?  The true answer may be something that is not an integer, so you may wish to convert to a cme257float.

In [ ]:
function convert(::Type{cme257float}, a::cme257int) 
    return cme257float(a.x)
end
@show a = cme257int(5)
@show convert(cme257float, a)
;

now since we can convert cme257ints to cme257floats, we can define division

In [ ]:
import Base./
function /(a::cme257float, b::cme257float)
    return cme257float(a.x / b.x) 
end

function /(a::cme257int, b::cme257int)
    return convert(cme257float, a) / convert(cme257float, b) 
end

a = cme257int(5)
b = cme257int(4)
@show a/b
;

Note that Julia's built in arithmetic types and operators already have conversion defined and implemented where necessary.  You mostly need to know about this if you're defining your own types.

## Modules

Modules are useful for packaging (modularizing) functionality that you create.  This allows you to re-use code, or automatically import functions that you like to use when you start a new Julia session.

In [ ]:
include("cme257mod.jl") # you need to provide an accurate absolute or relative path
cme257mod.speak()
@show x = cme257mod.ModType(5)
;

## Break for Worksheet 3